In [3]:
import numpy as np
import scipy.stats as stats
import pywt
import librosa
from scipy.stats import skew, kurtosis
import pandas as pd

# Load the audio file
audio_file = "F://Python all practice//310 Project//audio_sample.wav"
signal, sr = librosa.load(audio_file)

# Teager-Kaiser Energy Operator (TKEO) Features
tkeo = np.array([signal[i]**2 - signal[i-1] * signal[i+1] for i in range(1, len(signal)-1)])

# Wavelet Features
wavelet = 'db1'
level = 9
coeffs = pywt.wavedec(signal, wavelet, level=level)

# Calculate Fundamental Frequency (F0)
fmin = librosa.note_to_hz('C2')
fmax = librosa.note_to_hz('C7')
f0, _, _ = librosa.pyin(signal, fmin=fmin, fmax=fmax)
lowest_f0 = np.nanmin(f0) if np.any(~np.isnan(f0)) else 0  # Handling case where f0 might be all NaN

# MFCC and Delta Features
n_mfcc = 20
frame_size_ms = 25
hop_length_ms = 10
frame_size_samples = librosa.time_to_samples(frame_size_ms/1000)
hop_length_samples = librosa.time_to_samples(hop_length_ms/1000)
mfcc_features = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc, n_fft=frame_size_samples, hop_length=hop_length_samples)
delta_mfcc = librosa.feature.delta(mfcc_features)
delta_delta_mfcc = librosa.feature.delta(mfcc_features, order=2)

# Initialize an empty dictionary to hold all the features
features_dict = {
    "TKEO Mean": np.mean(tkeo),
    "TKEO Std Deviation": np.std(tkeo),
    "TKEO Max": np.max(tkeo),
    "TKEO Min": np.min(tkeo),
    "TKEO Skewness": skew(tkeo),
    "TKEO Kurtosis": kurtosis(tkeo),
    "Fundamental Frequency": lowest_f0,
    "ZCR": np.mean(librosa.feature.zero_crossing_rate(signal)),
    "Energy": np.sum(signal**2) / len(signal),
    "Amplitude": np.max(np.abs(signal)),
    "Spectral Centroid": np.mean(librosa.feature.spectral_centroid(y=signal, sr=sr)),
    "Spectral Rolloff": np.mean(librosa.feature.spectral_rolloff(y=signal, sr=sr)),
    "Spectral Flatness": np.mean(librosa.feature.spectral_flatness(y=signal)),
    "Spectral Bandwidth": np.mean(librosa.feature.spectral_bandwidth(y=signal, sr=sr))
}

# Add MFCC features and their deltas to the dictionary
for i in range(n_mfcc):
    features_dict[f"MFCC_{i+1}"] = mfcc_features[i].mean()
    features_dict[f"Delta MFCC_{i+1}"] = delta_mfcc[i].mean()
    features_dict[f"Delta-Delta MFCC_{i+1}"] = delta_delta_mfcc[i].mean()

# Add Wavelet features to the dictionary
for index, coeff in enumerate(coeffs):
    features_dict[f"Wavelet level {index+1} Energy"] = np.sum(np.square(coeff))
    features_dict[f"Wavelet level {index+1} Mean"] = np.mean(coeff)
    features_dict[f"Wavelet level {index+1} Std Deviation"] = np.std(coeff)
    features_dict[f"Wavelet level {index+1} Max"] = np.max(coeff)
    features_dict[f"Wavelet level {index+1} Min"] = np.min(coeff)
    features_dict[f"Wavelet level {index+1} Median"] = np.median(coeff)
    features_dict[f"Wavelet level {index+1} Skewness"] = skew(coeff)
    features_dict[f"Wavelet level {index+1} Kurtosis"] = kurtosis(coeff)

# Convert the dictionary to a pandas DataFrame
features_df = pd.DataFrame([features_dict])

# Save the DataFrame to a CSV file
features_df.to_csv("audio_features.csv", index=False)

print("Features successfully saved to audio_features.csv")


Features successfully saved to audio_features.csv
